<a href="https://colab.research.google.com/github/YounSooKimTech/Joy_Profiel_Photos/blob/main/Columbia_Meta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Meta Data for Columbia
## management, dro, marketing, accounting, and finance department

In [244]:
url_s = []
name_s =[]
title_s = []
department_s = []
prof_text_s =[]


In [245]:
# Get url for personal page

import pandas as pd
import requests
from bs4 import BeautifulSoup

departments = ["management", "dro", "marketing", "accounting", "finance"]

for department in departments:
  department_url = f"https://business.columbia.edu/faculty/divisions/{department}/faculty"
  response = requests.get(department_url)
  soup = BeautifulSoup(response.content, 'html.parser')
  #print(department_url, "\t", response)

  # Prof name in the department page
  name_tags = soup.find_all("h3", class_="m-listing-faculty__title")

  for tag in name_tags:
      name = tag.get_text().strip()
      new_name = name.lower().replace(" ", "-").replace(".", "").replace("(", "").replace(")", "")
      #print(new_name)

      # Error handling
      if new_name == "emmanuel-a-yimfor":
        new_name = "emmanuel-yimfor"
      elif new_name == "chris-lasala":
        new_name = "christopher-lasala"

      elif new_name == "lynne-sagalyn":
        prof_url = "https://business.columbia.edu/staff/people/lynne-sagalyn"
      elif new_name == "hannah-li":
        prof_url = "https://business.columbia.edu/staff/people/hannah-li"

      else:
        prof_url = f'https://business.columbia.edu/faculty/people/{new_name}'

        response = requests.get(prof_url)
        #print(prof_url, response)

      url_s.append(prof_url)

In [246]:
# From personal page, extract information

for url in url_s:
  response = requests.get(url)
  soup = BeautifulSoup(response.content, "html.parser")

  name = soup.find("h1", class_ = "o-profile__name").get_text().strip()

  info_tag = soup.find("div", class_="field field-person-title-affiliations field__items")
  title = info_tag.find("dt", class_ = "m-detail-meta__item-title").get_text().strip()
  department = info_tag.find("dd", class_="m-detail-meta__item-content").get_text().strip()

  prof_text = soup.find("div", class_ = "o-profile__content").get_text().strip()
  #print(prof_text)

  name_s.append(name)
  title_s.append(title)
  department_s.append(department)
  prof_text_s.append(prof_text)

In [247]:
df_meta = pd.DataFrame({"Name":name_s,
                        "Title":title_s,
                        "Department":department_s,
                        "URL":url_s,
                        "Prof_Text":prof_text_s})

df_meta.head()

,Name,Title,Department,URL,Prof_Text
0,Mabel Abraham,Barbara and Meyer Feldberg Associate Professor...,Management Division,https://business.columbia.edu/faculty/people/m...,Mabel Abraham is the Barbara and Meyer Feldber...
1,Eric Abrahamson,Hughie E. Mills Professor of Business,Management Division,https://business.columbia.edu/faculty/people/e...,"Professor Abrahamson studies the creation, spr..."
2,Modupe Akinola,Barbara and David Zalaznick Professor of Business,Management Division,https://business.columbia.edu/faculty/people/m...,Modupe Akinola is the Barbara and David Zalazn...
3,Daniel Ames,Ting Tsung and Wei Fong Chao Professor of Busi...,Management Division,https://business.columbia.edu/faculty/people/d...,Professor Ames's research focuses on social ju...
4,Joel Brockner,Phillip Hettleman Professor of Business,Management Division,https://business.columbia.edu/faculty/people/j...,Within the broader field of organizational beh...


In [248]:
df_meta_prof = df_meta[~df_meta["Title"].str.contains("Practice|Lecturer|Instructor")]

df_meta_prof.head()

,Name,Title,Department,URL,Prof_Text
0,Mabel Abraham,Barbara and Meyer Feldberg Associate Professor...,Management Division,https://business.columbia.edu/faculty/people/m...,Mabel Abraham is the Barbara and Meyer Feldber...
1,Eric Abrahamson,Hughie E. Mills Professor of Business,Management Division,https://business.columbia.edu/faculty/people/e...,"Professor Abrahamson studies the creation, spr..."
2,Modupe Akinola,Barbara and David Zalaznick Professor of Business,Management Division,https://business.columbia.edu/faculty/people/m...,Modupe Akinola is the Barbara and David Zalazn...
3,Daniel Ames,Ting Tsung and Wei Fong Chao Professor of Busi...,Management Division,https://business.columbia.edu/faculty/people/d...,Professor Ames's research focuses on social ju...
4,Joel Brockner,Phillip Hettleman Professor of Business,Management Division,https://business.columbia.edu/faculty/people/j...,Within the broader field of organizational beh...


In [249]:
import numpy as np

conditions = [
    df_meta_prof["Title"].str.contains("Emeritus"),
    df_meta_prof["Title"].str.contains("Emerita"),
    df_meta_prof["Title"].str.contains("Assistant"),
    df_meta_prof["Title"].str.contains("Associate")
]

choices = ["Emeritus", "Emeritus", "Assistant", "Associate"]

df_meta_prof["Rank"] = np.select(conditions, choices, default="Full")

df_meta_prof["University"] = "Columbia"

#df_meta_prof.to_csv("Columbia_meta_1.csv", index=False)

<ipython-input-249-c19a1f7d136a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_prof["Rank"] = np.select(conditions, choices, default="Full")
<ipython-input-249-c19a1f7d136a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_prof["University"] = "Columbia"


In [250]:
import spacy
nlp = spacy.load('en_core_web_sm')

pronouns_list = []

for text in df_meta_prof['Prof_Text']:
    doc = nlp(text)
    pronouns = [token.text.lower() for token in doc if token.pos_ == 'PRON' and token.text.lower() in ['he', 'she', 'her', 'his']]
    pronouns_list.append(pronouns)

df_meta_prof['Pronouns'] = pronouns_list

def determine_sex(pronouns):
    if 'he' in pronouns or 'his' in pronouns:
        return 'Male'
    elif 'she' in pronouns or 'her' in pronouns:
        return 'Female'
    else:
        return 'Unknown'

df_meta_prof['Sex'] = df_meta_prof['Pronouns'].apply(determine_sex)

df_meta_prof[["Pronouns", "Sex"]]


<ipython-input-250-51d2cdee2884>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_prof['Pronouns'] = pronouns_list
<ipython-input-250-51d2cdee2884>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_prof['Sex'] = df_meta_prof['Pronouns'].apply(determine_sex)


,Pronouns,Sex
0,"[she, she, her, her, she, her]",Female
1,"[he, his, he, he, his, he]",Male
2,"[she, she, she]",Female
3,"[he, he]",Male
4,"[his, he, he]",Male
...,...,...
138,"[she, she, she, her]",Female
139,"[he, he, he, he, his, he, he, he]",Male
140,"[he, he, his, he, his, he, his]",Male
141,"[his, his, he]",Male


In [251]:
df_meta_prof_1 = df_meta_prof[["University", "Name", "Department", "Rank", "Sex", "Title", "Pronouns", "URL"]]

df_meta_prof_1[df_meta_prof_1["Sex"] == "Unknown"]

,University,Name,Department,Rank,Sex,Title,Pronouns,URL
57,Columbia,Martin Starr,"Decision, Risk, and Operations Division",Emeritus,Unknown,Professor Emeritus of Management Science and O...,[],https://business.columbia.edu/faculty/people/m...
102,Columbia,Syrena Shirley,Accounting Division,Assistant,Unknown,Assistant Professor of Business,[],https://business.columbia.edu/faculty/people/s...
124,Columbia,Xuelin Li,Finance Division,Assistant,Unknown,Assistant Professor of Business,[],https://business.columbia.edu/faculty/people/x...
128,Columbia,Bruno Pellegrino,Finance Division,Assistant,Unknown,Assistant Professor of Business,[],https://business.columbia.edu/faculty/people/b...


In [252]:
# https://millstein.law.columbia.edu/people/syrena-shirley
# https://sc.edu/study/colleges_schools/moore/directory/li_xuelin.php
# Martin Starr	 <- no photos
# Bruno Pellegrino	<- no additional info about the pronouns.

df_meta_prof_1.loc[df_meta_prof_1["Name"] == "Syrena Shirley", "Sex"] = "Female"
df_meta_prof_1.loc[df_meta_prof_1["Name"] == "Xuelin Li", "Sex"] = "Male"

<ipython-input-252-6ebd52381bc1>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_prof_1.loc[df_meta_prof_1["Name"] == "Syrena Shirley", "Sex"] = "Female"
<ipython-input-252-6ebd52381bc1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_prof_1.loc[df_meta_prof_1["Name"] == "Xuelin Li", "Sex"] = "Male"


In [253]:
df_meta_prof_1[df_meta_prof_1["Sex"] == "Unknown"]

,University,Name,Department,Rank,Sex,Title,Pronouns,URL
57,Columbia,Martin Starr,"Decision, Risk, and Operations Division",Emeritus,Unknown,Professor Emeritus of Management Science and O...,[],https://business.columbia.edu/faculty/people/m...
128,Columbia,Bruno Pellegrino,Finance Division,Assistant,Unknown,Assistant Professor of Business,[],https://business.columbia.edu/faculty/people/b...


# Economics Division

In [254]:
url_s = []
name_s =[]
title_s = []
department_s = []
prof_text_s =[]

In [255]:
url = "https://business.columbia.edu/directory/departments/economics"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
#print(url, "\t", response)

name_tags = soup.find_all("h3", class_="m-listing-faculty__title")

for a_name_tag in name_tags:
  name = a_name_tag.get_text().strip()

  new_name = name.lower().replace(" ", "-").replace(".", "")

  if new_name == "cristóbal-otero":
    new_name = "cristobal-otero-ruiz-tagle"

  elif new_name == "thomas--prusa":
    new_name = "thomas-prusa"
  elif new_name == "nishat-a-juy":
    new_name = "nishat-juy"


  url = f"https://business.columbia.edu/faculty/people/{new_name}"
  response = requests.get(url)

  if response.status_code == 404:
    url = f"https://business.columbia.edu/staff/people/{new_name}"
    response = requests.get(url)
  else:
    url = url
    response = response

  #print(url, response)
  url_s.append(url)


In [256]:
# From personal page, extract information

for url in url_s:
  response = requests.get(url)
  soup = BeautifulSoup(response.content, "html.parser")

  name = soup.find("h1", class_ = "o-profile__name").get_text().strip()
  name_s.append(name)

  info_tag = soup.find("div", class_="field field-person-title-affiliations field__items")
  title = info_tag.find("dt", class_ = "m-detail-meta__item-title").get_text().strip()
  department = info_tag.find("dd", class_="m-detail-meta__item-content").get_text().strip()

  prof_text = soup.find("div", class_ = "o-profile__content").get_text().strip()
  #print(prof_text)


  title_s.append(title)
  department_s.append(department)
  prof_text_s.append(prof_text)

  '''
  print(name)
  print(title)
  print(department, "\n")
  '''

In [257]:
df_meta = pd.DataFrame({"Name":name_s,
                        "Title":title_s,
                        "Department":department_s,
                        "URL":url_s,
                        "Prof_Text":prof_text_s})

#df_meta["Title"].value_counts()


In [258]:
df_meta.loc[df_meta["Name"] == "Eli Noam", "Title"] = "Emeritus"
df_meta.loc[df_meta["Name"] == "Jason Buckweitz", "Title"] = "Adjunct"
df_meta.loc[df_meta["Name"] == "R.A. Farrokhnia", "Title"] = "Adjunct"

In [259]:
df_meta[df_meta["Title"].str.contains("Director")]

,Name,Title,Department,URL,Prof_Text
5,Jessica Brucas,Associate Director,Finance Division,https://business.columbia.edu/staff/people/jes...,
21,Andrea Gurwitt,Editor and Communications Director,Economics Division,https://business.columbia.edu/staff/people/and...,
33,Rocio Martinez,Assistant Director,Economics Division,https://business.columbia.edu/staff/people/roc...,
62,Megan Winborne,Associate Director,Economics Division,https://business.columbia.edu/staff/people/meg...,
65,Michelle Zern,Director of Academic Administration,Finance Division,https://business.columbia.edu/staff/people/mic...,


In [260]:
df_meta_prof = df_meta[~df_meta["Title"].str.contains("Practice|Lecturer|Instructor|Staff|Adjunct|Postdoctoral|Administrative|Executive Assistant|Director|Program Coordinator")]

df_meta_prof["Title"].value_counts()

Assistant Professor of Business                                                          5
Associate Professor of Business                                                          2
Merrill Lynch Professor of Workforce Transformation                                      1
Class of 1967 Associate Professor of Business                                            1
MUTB Professor of International Business                                                 1
Faculty Emeriti                                                                          1
N T  Wang Professor of Chinese Business and Economy                                      1
Professor                                                                                1
Franklin Pitcher Johnson Jr. Professor of Finance and Economics                          1
Carson Family Professor of Business; Chair of Economics Division                         1
Professor Emeritus of Business                                                           1

In [261]:
import numpy as np

conditions = [
    df_meta_prof["Title"].str.contains("Emeritus|Emerita|Emeriti"),
    df_meta_prof["Title"].str.contains("Assistant"),
    df_meta_prof["Title"].str.contains("Associate")
]

choices = ["Emeritus", "Assistant", "Associate"]

df_meta_prof["Rank"] = np.select(conditions, choices, default="Full")

df_meta_prof["University"] = "Columbia"


<ipython-input-261-213a92a4f6a1>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_prof["Rank"] = np.select(conditions, choices, default="Full")
<ipython-input-261-213a92a4f6a1>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_prof["University"] = "Columbia"


In [262]:
import spacy
nlp = spacy.load('en_core_web_sm')

pronouns_list = []

for text in df_meta_prof['Prof_Text']:
    doc = nlp(text)
    pronouns = [token.text.lower() for token in doc if token.pos_ == 'PRON' and token.text.lower() in ['he', 'she', 'her', 'his']]
    pronouns_list.append(pronouns)

df_meta_prof['Pronouns'] = pronouns_list

def determine_sex(pronouns):
    if 'he' in pronouns or 'his' in pronouns:
        return 'Male'
    elif 'she' in pronouns or 'her' in pronouns:
        return 'Female'
    else:
        return 'Unknown'

df_meta_prof['Sex'] = df_meta_prof['Pronouns'].apply(determine_sex)

df_meta_prof[["Pronouns", "Sex"]]


<ipython-input-262-51d2cdee2884>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_prof['Pronouns'] = pronouns_list
<ipython-input-262-51d2cdee2884>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_prof['Sex'] = df_meta_prof['Pronouns'].apply(determine_sex)


,Pronouns,Sex
2,"[she, she, she]",Female
3,"[her, her, she]",Female
8,[],Unknown
11,"[he, he, his, he, he]",Male
12,"[he, his]",Male
13,"[she, her, her, she]",Female
15,"[he, his, he, his]",Male
22,"[he, he, he, he, he, he, he]",Male
24,"[he, his, he]",Male
28,[],Unknown


In [263]:
df_meta_prof.head()

,Name,Title,Department,URL,Prof_Text,Rank,University,Pronouns,Sex
2,Ann Bartel,Merrill Lynch Professor of Workforce Transform...,Economics Division,https://business.columbia.edu/faculty/people/a...,Professor Bartel is the Merrill Lynch Professo...,Full,Columbia,"[she, she, she]",Female
3,Laura Boudreau,Assistant Professor of Business,Economics Division,https://business.columbia.edu/faculty/people/l...,Laura Boudreau is an Assistant Director at Col...,Assistant,Columbia,"[her, her, she]",Female
8,Abby Cohen,Professor of Business,Economics Division,https://business.columbia.edu/faculty/people/a...,,Full,Columbia,[],Unknown
11,Wouter Dessein,Eli Ginzberg Professor of Finance and Economics,Economics Division,https://business.columbia.edu/faculty/people/w...,Wouter Dessein is the Eli Ginzberg Professor o...,Full,Columbia,"[he, he, his, he, he]",Male
12,John Donaldson,Mario J. Gabelli Professor of Finance,Economics Division,https://business.columbia.edu/faculty/people/j...,Professor Donaldson teaches courses in basic f...,Full,Columbia,"[he, his]",Male


In [264]:
df_meta_prof_2 = df_meta_prof[["University", "Name", "Department", "Rank", "Sex", "Title", "Pronouns", "URL"]]

df_meta_prof_2[df_meta_prof_2["Sex"]=="Unknown"]

,University,Name,Department,Rank,Sex,Title,Pronouns,URL
8,Columbia,Abby Cohen,Economics Division,Full,Unknown,Professor of Business,[],https://business.columbia.edu/faculty/people/a...
28,Columbia,Nathaniel Leff,Economics Division,Emeritus,Unknown,Faculty Emeritus,[],https://business.columbia.edu/faculty/people/n...
32,Columbia,Kristina Manysheva,Economics Division,Assistant,Unknown,Assistant Professor of Business,[],https://business.columbia.edu/faculty/people/k...
37,Columbia,Eli Noam,Economics Division,Emeritus,Unknown,Emeritus,[],https://business.columbia.edu/faculty/people/e...
58,Columbia,Conor Walsh,Economics Division,Assistant,Unknown,Assistant Professor of Business,[],https://business.columbia.edu/faculty/people/c...


In [265]:
# https://en.wikipedia.org/wiki/Abby_Joseph_Cohen
# https://prabook.com/web/nathaniel.leff/953812
# https://cepr.org/about/people/kristina-manysheva
# https://en.wikipedia.org/wiki/Eli_Noam
# https://wyss.harvard.edu/team/associate-faculty/conor-walsh/

df_meta_prof_2.loc[df_meta_prof_2["Name"]=="Abby Cohen", "Sex"] = "Female"
df_meta_prof_2.loc[df_meta_prof_2["Name"]=="Nathaniel Leff", "Sex"] = "Male"
df_meta_prof_2.loc[df_meta_prof_2["Name"]=="Kristina Manysheva", "Sex"] = "Female"
df_meta_prof_2.loc[df_meta_prof_2["Name"]=="Eli Noam", "Sex"] = "Male"
df_meta_prof_2.loc[df_meta_prof_2["Name"]=="Conor Walsh", "Sex"] = "Male"


<ipython-input-265-85ab2be5f9d4>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_prof_2.loc[df_meta_prof_2["Name"]=="Abby Cohen", "Sex"] = "Female"
<ipython-input-265-85ab2be5f9d4>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_prof_2.loc[df_meta_prof_2["Name"]=="Nathaniel Leff", "Sex"] = "Male"
<ipython-input-265-85ab2be5f9d4>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_prof_2.loc[df_meta_prof_2["Name"]

In [266]:
df_meta_prof_2[df_meta_prof_2["Sex"]=="Unknown"]

,University,Name,Department,Rank,Sex,Title,Pronouns,URL


# Merge departments and econ

In [267]:
df_meta_prof_1.head()

,University,Name,Department,Rank,Sex,Title,Pronouns,URL
0,Columbia,Mabel Abraham,Management Division,Associate,Female,Barbara and Meyer Feldberg Associate Professor...,"[she, she, her, her, she, her]",https://business.columbia.edu/faculty/people/m...
1,Columbia,Eric Abrahamson,Management Division,Full,Male,Hughie E. Mills Professor of Business,"[he, his, he, he, his, he]",https://business.columbia.edu/faculty/people/e...
2,Columbia,Modupe Akinola,Management Division,Full,Female,Barbara and David Zalaznick Professor of Business,"[she, she, she]",https://business.columbia.edu/faculty/people/m...
3,Columbia,Daniel Ames,Management Division,Full,Male,Ting Tsung and Wei Fong Chao Professor of Busi...,"[he, he]",https://business.columbia.edu/faculty/people/d...
4,Columbia,Joel Brockner,Management Division,Full,Male,Phillip Hettleman Professor of Business,"[his, he, he]",https://business.columbia.edu/faculty/people/j...


In [268]:
df_meta_prof_2.head()

,University,Name,Department,Rank,Sex,Title,Pronouns,URL
2,Columbia,Ann Bartel,Economics Division,Full,Female,Merrill Lynch Professor of Workforce Transform...,"[she, she, she]",https://business.columbia.edu/faculty/people/a...
3,Columbia,Laura Boudreau,Economics Division,Assistant,Female,Assistant Professor of Business,"[her, her, she]",https://business.columbia.edu/faculty/people/l...
8,Columbia,Abby Cohen,Economics Division,Full,Female,Professor of Business,[],https://business.columbia.edu/faculty/people/a...
11,Columbia,Wouter Dessein,Economics Division,Full,Male,Eli Ginzberg Professor of Finance and Economics,"[he, he, his, he, he]",https://business.columbia.edu/faculty/people/w...
12,Columbia,John Donaldson,Economics Division,Full,Male,Mario J. Gabelli Professor of Finance,"[he, his]",https://business.columbia.edu/faculty/people/j...


In [269]:
df_columbia = pd.concat([df_meta_prof_1, df_meta_prof_2], ignore_index=True)

print(df_columbia.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   University  153 non-null    object
 1   Name        153 non-null    object
 2   Department  153 non-null    object
 3   Rank        153 non-null    object
 4   Sex         153 non-null    object
 5   Title       153 non-null    object
 6   Pronouns    153 non-null    object
 7   URL         153 non-null    object
dtypes: object(8)
memory usage: 9.7+ KB
None


In [274]:
df_columbia[df_columbia.duplicated(subset="URL")]

,University,Name,Department,Rank,Sex,Title,Pronouns,URL
122,Columbia,Kairong Xiao,Finance Division,Associate,Male,Roger F. Murray Associate Professor of Business,"[his, his, he]",https://business.columbia.edu/faculty/people/k...
143,Columbia,Jesse Schreger,Economics Division,Associate,Male,Associate Professor of Business,"[his, his, he, his, his]",https://business.columbia.edu/faculty/people/j...


In [279]:
df_columbia[df_columbia["Name"] == "Kairong Xiao"]

,University,Name,Department,Rank,Sex,Title,Pronouns,URL
121,Columbia,Kairong Xiao,Finance Division,Associate,Male,Roger F. Murray Associate Professor of Business,"[his, his, he]",https://business.columbia.edu/faculty/people/k...
122,Columbia,Kairong Xiao,Finance Division,Associate,Male,Roger F. Murray Associate Professor of Business,"[his, his, he]",https://business.columbia.edu/faculty/people/k...


In [280]:
df_columbia[df_columbia["Name"] == "Jesse Schreger"]

,University,Name,Department,Rank,Sex,Title,Pronouns,URL
113,Columbia,Jesse Schreger,Economics Division,Associate,Male,Associate Professor of Business,"[his, his, he, his, his]",https://business.columbia.edu/faculty/people/j...
143,Columbia,Jesse Schreger,Economics Division,Associate,Male,Associate Professor of Business,"[his, his, he, his, his]",https://business.columbia.edu/faculty/people/j...


In [271]:
df_columbia_no_dupe = df_columbia.drop_duplicates(subset=["URL"])
print(df_columbia_no_dupe.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151 entries, 0 to 152
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   University  151 non-null    object
 1   Name        151 non-null    object
 2   Department  151 non-null    object
 3   Rank        151 non-null    object
 4   Sex         151 non-null    object
 5   Title       151 non-null    object
 6   Pronouns    151 non-null    object
 7   URL         151 non-null    object
dtypes: object(8)
memory usage: 10.6+ KB
None


In [281]:
#df_columbia_no_dupe.to_csv("Meta_Prof_Columbia.csv", index=False)

In [282]:
df_columbia_no_dupe["Sex"].value_counts()

Male       108
Female      41
Unknown      2
Name: Sex, dtype: int64

In [291]:
df_columbia_no_dupe["Department"].value_counts()

Finance Division                           32
Management Division                        30
Economics Division                         30
Marketing Division                         24
Decision, Risk, and Operations Division    19
Accounting Division                        14
Dean's Office                               2
Name: Department, dtype: int64

In [288]:
round(df_columbia_no_dupe["Rank"].value_counts()/len(df_columbia_no_dupe)*100, 2)

Full         46.36
Assistant    22.52
Associate    16.56
Emeritus     14.57
Name: Rank, dtype: float64